In [1]:
from keras.models import load_model
from keras.preprocessing import image
import cv2
import numpy as np

Using TensorFlow backend.


In [2]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
classifier = load_model('Emotion_little_vgg.h5')

In [3]:
class_labels = ['Angry','Happy','Neutral','Sad','Surprise']

In [4]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    
    ret, frame = cap.read()
    
    if ret == False:
        continue
    
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # getting gray scale frame
    faces = face_classifier.detectMultiScale(gray,1.3,6)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48))
        
        # If there is a face in the frame
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float')/255.0
            roi = image.img_to_array(roi)
            roi = np.expand_dims(roi,axis=0) # Adds a 1-sized dimension at row.
            
            # Making prediction on ROI
            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            label_position = (x,y)
            
            cv2.putText(gray,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),3)
        else:
            cv2.putText(gray,"No Face Found",label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),3)
        
        cv2.imshow('Emotion Detector',gray)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()